In [15]:
import requests
import time
import csv
import os
import json


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from imdb import IMDb

In [17]:
TMDB_API_KEY = '2762321c463faabbba42b561409706aa'  # Replace this with your actual TMDB API key
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

In [18]:
# Module 1: Fetch basic movie list from Wikipedia
# This function scrapes Wikipedia to get a list of top highest-grossing films.
# It returns a DataFrame with columns: Rank, Title, Worldwide_Gross, Year, Wiki_Link
def fetch_highest_grossing_films(num_movies=50):
    """
    Scrape the list of highest-grossing films from Wikipedia.

    This function:
    1. Sends a request to the Wikipedia URL.
    2. Parses the HTML to find the table of highest-grossing films.
    3. Extracts data from the table rows (skipping the header).
    4. Limits the results to the specified number of movies.

    Args:
        num_movies (int): How many top movies to fetch (default is 50 to avoid too many requests).

    Returns:
        pd.DataFrame: A table with basic movie info.
    """
    # Send HTTP request to Wikipedia
    url = WIKI_URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the specific table on the page (it's the first 'wikitable sortable')
    table = soup.find('table', {'class': 'wikitable sortable'})
    # Get all rows in the table
    rows = table.find_all('tr')

    # List to store movie data
    data = []
    # Loop through rows starting from index 1 (skip header) up to num_movies + 1
    for row in rows[1:num_movies+1]:
        # Get all columns (td tags) in the row
        cols = row.find_all('td')
        if len(cols) >= 4:  # Ensure it's a valid row with at least 4 columns
            rank = cols[0].text.strip()  # Clean text: remove extra spaces
            title = cols[1].text.strip()
            gross = cols[2].text.strip().replace('$', '').replace(',', '')  # Remove $ and commas for numeric value
            year = cols[3].text.strip()
            # Get the Wikipedia link if available
            link = "https://en.wikipedia.org" + cols[1].find('a')['href'] if cols[1].find('a') else None
            # Add to data list as a dictionary
            data.append({
                'Rank': rank,
                'Title': title,
                'Worldwide_Gross': gross,
                'Year': year,
                'Wiki_Link': link
            })

    # Convert list of dicts to a pandas DataFrame
    return pd.DataFrame(data)

    #this was later discarded due to data not being parsed from wikipedia.

In [19]:
# --- Exploratory TMDB fetch (not used in final pipeline) ---
# def fetch_tmdb_test_version_1():


# Module 2: Fetch details from TMDB for a single movie
# This function queries TMDB API for one movie's details using title and year.
def fetch_tmdb_details(title, year):
    """
    Fetch movie details from TMDB API for a single movie.

    This function:
    1. Searches TMDB for the movie by title and year.
    2. If found, gets the movie ID.
    3. Fetches detailed info including credits (cast and crew).
    4. Extracts features like genre, director, actors, etc.

    Args:
        title (str): Movie title.
        year (str): Release year.

    Returns:
        dict: A dictionary with TMDB features (or 'N/A' if not found).
    """
    # Default dictionary with 'N/A' values
    movie_data = {
        'Genre': 'N/A',
        'Director': 'N/A',
        'Actors': 'N/A',
        'Release_Date': 'N/A',
        'Runtime': 'N/A',
        'Budget': 'N/A',
        'Revenue': 'N/A',
        'Vote_Average': 'N/A'
    }

    try:
        # Construct search URL with API key, title, and year
        search_url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={title}&year={year}"
        # Get response and parse as JSON
        search_response = requests.get(search_url).json()

        # If search results exist
        if search_response['results']:
            # Take the first result's ID (most relevant)
            movie_id = search_response['results'][0]['id']
            # Construct details URL with credits appended
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&append_to_response=credits"
            details_response = requests.get(details_url).json()

            # Extract genres as comma-separated string
            movie_data['Genre'] = ', '.join([genre['name'] for genre in details_response.get('genres', [])])

            # Find director from crew list
            director = next((member['name'] for member in details_response['credits'].get('crew', []) if member['job'] == 'Director'), 'N/A')
            movie_data['Director'] = director

            # Get top 5 actors from cast
            actors = details_response['credits'].get('cast', [])[:5]
            movie_data['Actors'] = ', '.join([actor['name'] for actor in actors])

            # Other details
            movie_data['Release_Date'] = details_response.get('release_date', 'N/A')
            movie_data['Runtime'] = details_response.get('runtime', 'N/A')
            movie_data['Budget'] = details_response.get('budget', 'N/A')
            movie_data['Revenue'] = details_response.get('revenue', 'N/A')
            movie_data['Vote_Average'] = details_response.get('vote_average', 'N/A')

        # Delay to avoid hitting API rate limits
        time.sleep(1)
        return movie_data
    except Exception as e:
        # Print error if something goes wrong
        print(f"Error with TMDB for {title}: {e}")
        return movie_data

# This function applies fetch_tmdb_details to all rows in a DataFrame
def enhance_with_tmdb(df):
    """
    Enhance the entire DataFrame with TMDB details.

    This loops through each row, fetches TMDB data, and adds it to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame with 'Title' and 'Year'.

    Returns:
        pd.DataFrame: DataFrame with added TMDB columns.
    """
    # List to store TMDB data for each movie
    tmdb_data = []
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        details = fetch_tmdb_details(row['Title'], row['Year'])
        tmdb_data.append(details)

    # Convert list to DataFrame and concatenate horizontally
    tmdb_df = pd.DataFrame(tmdb_data)
    return pd.concat([df, tmdb_df], axis=1)

In [20]:
# --- Exploratory TMDB fetch (not used in final pipeline) ---
# def fetch_tmdb_test_version_2():

# Import necessary libraries
# - requests: For making HTTP requests to APIs
# - pandas: For handling data in DataFrames (like tables)
# - time: For adding delays to avoid overwhelming servers
# - IMDb: From imdbpy library, for accessing IMDB data (install with pip install imdbpy) - optional
import requests
import pandas as pd
import time
from imdb import IMDb

# Configuration settings
# - TMDB_API_KEY: Provided TMDB API key
TMDB_API_KEY = '2762321c463faabbba42b561409706aa'

# Module 1: Fetch top movies from TMDB (sorted by revenue.desc)
# This function uses TMDB's discover/movie endpoint to get top-grossing films.
# It returns a DataFrame with columns: Title, Year, TMDB_ID
def fetch_top_movies_from_tmdb(num_movies=50, min_year=None):
    """
    Fetch top-grossing movies from TMDB using discover/movie sorted by revenue.desc.

    This function:
    1. Constructs the API URL with sort_by=revenue.desc and pagination.
    2. Optionally filters by minimum year (e.g., primary_release_year.gte=2010).
    3. Extracts title, year (from release_date), and ID for each movie.
    4. Stops once num_movies is reached.

    Args:
        num_movies (int): How many top movies to fetch (default 50).
        min_year (int): Optional minimum release year (e.g., 2010 for recent data).

    Returns:
        pd.DataFrame: A table with basic movie info (Title, Year, TMDB_ID).
    """
    movies = []
    page = 1
    while len(movies) < num_movies:
        # Construct URL with sort_by=revenue.desc
        url = f"https://api.themoviedb.org/3/discover/movie?api_key={TMDB_API_KEY}&sort_by=revenue.desc&page={page}"
        if min_year:
            url += f"&primary_release_year.gte={min_year}"  # Filter for recent years if specified
        response = requests.get(url).json()

        if 'results' not in response or not response['results']:
            break  # No more results

        for movie in response['results']:
            year = movie.get('release_date', '')[:4]  # Extract year from release_date
            if year:  # Skip if no release date
                movies.append({
                    'Title': movie['title'],
                    'Year': year,
                    'TMDB_ID': movie['id']
                })
            if len(movies) >= num_movies:
                break

        page += 1
        time.sleep(1)  # Delay to avoid rate limits

    if not movies:
        print("No movies fetched from TMDB. Check API key or parameters.")
        return pd.DataFrame()

    return pd.DataFrame(movies)

# Module 2: Fetch details from TMDB for a single movie using ID
# This function queries TMDB's movie/{id} endpoint for details.
def fetch_tmdb_details(movie_id):
    """
    Fetch movie details from TMDB API for a single movie using its ID.

    This function:
    1. Fetches detailed info including credits (cast and crew).
    2. Extracts features like genre, director, actors, release_date, runtime, budget, revenue (as Worldwide_Gross), vote_average.

    Args:
        movie_id (int): TMDB movie ID.

    Returns:
        dict: A dictionary with TMDB features (or 'N/A' if not found).
    """
    # Default dictionary with 'N/A' values
    movie_data = {
        'Genre': 'N/A',
        'Director': 'N/A',
        'Actors': 'N/A',
        'Release_Date': 'N/A',
        'Runtime': 'N/A',
        'Budget': 'N/A',
        'Worldwide_Gross': 'N/A',  # Use revenue as gross
        'Vote_Average': 'N/A'
    }

    try:
        # Construct details URL with credits appended
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&append_to_response=credits"
        details_response = requests.get(details_url).json()

        if 'status_code' in details_response and details_response['status_code'] != 200:
            raise Exception(details_response.get('status_message', 'Unknown error'))

        # Extract genres as comma-separated string
        movie_data['Genre'] = ', '.join([genre['name'] for genre in details_response.get('genres', [])])

        # Find director from crew list
        director = next((member['name'] for member in details_response['credits'].get('crew', []) if member['job'] == 'Director'), 'N/A')
        movie_data['Director'] = director

        # Get top 5 actors from cast
        actors = details_response['credits'].get('cast', [])[:5]
        movie_data['Actors'] = ', '.join([actor['name'] for actor in actors])

        # Other details
        movie_data['Release_Date'] = details_response.get('release_date', 'N/A')
        movie_data['Runtime'] = details_response.get('runtime', 'N/A')
        movie_data['Budget'] = details_response.get('budget', 'N/A')
        movie_data['Worldwide_Gross'] = details_response.get('revenue', 'N/A')  # Revenue as gross
        movie_data['Vote_Average'] = details_response.get('vote_average', 'N/A')

        time.sleep(1)  # Delay to avoid hitting API rate limits
        return movie_data
    except Exception as e:
        print(f"Error with TMDB for movie ID {movie_id}: {e}")
        return movie_data

# This function applies fetch_tmdb_details to all rows in a DataFrame using TMDB_ID
def enhance_with_tmdb(df):
    """
    Enhance the entire DataFrame with TMDB details using TMDB_ID.

    This loops through each row, fetches details by ID, and adds them to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame with 'TMDB_ID'.

    Returns:
        pd.DataFrame: DataFrame with added TMDB columns.
    """
    tmdb_data = []
    for _, row in df.iterrows():
        details = fetch_tmdb_details(row['TMDB_ID'])
        tmdb_data.append(details)

    tmdb_df = pd.DataFrame(tmdb_data)
    return pd.concat([df, tmdb_df], axis=1)

# Module 3: Fetch IMDB details for a single movie (optional)
def fetch_imdb_details(title):
    """
    Fetch IMDB details for a single movie.

    This uses the imdbpy library to search IMDB.

    Args:
        title (str): Movie title.

    Returns:
        dict: Dictionary with IMDB rating and votes (or 'N/A').
    """
    try:
        ia = IMDb()
        movies = ia.search_movie(title)
        if movies:
            movie = ia.get_movie(movies[0].movieID)
            return {
                'IMDB_Rating': movie.get('rating', 'N/A'),
                'Votes': movie.get('votes', 'N/A')
            }
        time.sleep(1)
        return {'IMDB_Rating': 'N/A', 'Votes': 'N/A'}
    except Exception as e:
        print(f"Error with IMDB for {title}: {e}")
        return {'IMDB_Rating': 'N/A', 'Votes': 'N/A'}

# This function applies fetch_imdb_details to all rows
def enhance_with_imdb(df):
    """
    Enhance the DataFrame with IMDB details.

    Args:
        df (pd.DataFrame): Input DataFrame with 'Title'.

    Returns:
        pd.DataFrame: DataFrame with added IMDB columns.
    """
    imdb_data = []
    for _, row in df.iterrows():
        details = fetch_imdb_details(row['Title'])
        imdb_data.append(details)

    imdb_df = pd.DataFrame(imdb_data)
    return pd.concat([df, imdb_df], axis=1)

# Module 4: Clean the data
# Converts financial columns to numeric values in millions.
def clean_data(df):
    """
    Clean the DataFrame by converting financials to numbers (in millions).

    This handles:
    - Converting Worldwide_Gross, Budget, and Revenue (if present) to floats divided by 1,000,000.
    - Handles 'N/A' as NaN.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    df['Worldwide_Gross'] = pd.to_numeric(df['Worldwide_Gross'], errors='coerce') / 1000000
    df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce') / 1000000
    return df

# Module 5: Build the full fact table
# This orchestrates everything: fetch top movies, enhance with details, clean, save.
def build_fact_table(num_movies=50, min_year=2010, use_imdb=True):
    """
    Build the complete fact table using only TMDB.

    Steps:
    1. Fetch top movies from TMDB (sorted by revenue.desc, optional min_year filter).
    2. Add TMDB details using movie IDs.
    3. Optionally add IMDB details.
    4. Clean the data.
    5. Save to CSV.

    Args:
        num_movies (int): Number of movies to process.
        min_year (int): Minimum release year (default 2010 for recent data; set to None for all time).
        use_imdb (bool): Whether to include IMDB data (default True).

    Returns:
        pd.DataFrame: The final fact table.
    """
    # Step 1: Get top movies from TMDB
    df = fetch_top_movies_from_tmdb(num_movies, min_year)
    if df.empty:
        return df
    # Step 2: Add TMDB details
    df = enhance_with_tmdb(df)
    # Step 3: Add IMDB if requested
    if use_imdb:
        df = enhance_with_imdb(df)
    # Step 4: Clean
    df = clean_data(df)

    # Save the DataFrame to a CSV file
    df.to_csv('movie_fact_table.csv', index=False)
    print("Fact table saved to movie_fact_table.csv")
    return df

# Run the script when executed directly
# This is the entry point: calls build_fact_table and prints the first few rows
# Adjust min_year as needed (e.g., None for all time, or 2000 for broader data)
if __name__ == "__main__":
    fact_table = build_fact_table(min_year=2010)  # Example: data from 2010 onward
    print(fact_table.head())

Fact table saved to movie_fact_table.csv
                      Title  Year  TMDB_ID  \
0                    Avatar  2009    19995   
1         Avengers: Endgame  2019   299534   
2  Avatar: The Way of Water  2022    76600   
3                   Titanic  1997      597   
4                  Ne Zha 2  2025   980477   

                                         Genre       Director  \
0  Action, Adventure, Fantasy, Science Fiction  James Cameron   
1           Adventure, Science Fiction, Action  Anthony Russo   
2           Action, Adventure, Science Fiction  James Cameron   
3                               Drama, Romance  James Cameron   
4        Animation, Fantasy, Adventure, Action        Jiao Zi   

                                              Actors Release_Date  Runtime  \
0  Sam Worthington, Zoe Saldaña, Sigourney Weaver...   2009-12-15      162   
1  Robert Downey Jr., Chris Evans, Mark Ruffalo, ...   2019-04-24      181   
2  Sam Worthington, Zoe Saldaña, Sigourney Weaver...   20

In [21]:
fact_table.head()

,Title,Year,TMDB_ID,Genre,Director,Actors,Release_Date,Runtime,Budget,Worldwide_Gross,Vote_Average,IMDB_Rating,Votes
0,Avatar,2009,19995,"Action, Adventure, Fantasy, Science Fiction",James Cameron,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...",2009-12-15,162,237.0,2923.706026,7.594,N/A,N/A
1,Avengers: Endgame,2019,299534,"Adventure, Science Fiction, Action",Anthony Russo,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",2019-04-24,181,356.0,2799.439100,8.237,N/A,N/A
2,Avatar: The Way of Water,2022,76600,"Action, Adventure, Science Fiction",James Cameron,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...",2022-12-14,192,350.0,2353.096253,7.609,N/A,N/A
3,Titanic,1997,597,"Drama, Romance",James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",1997-11-18,194,200.0,2264.162353,7.903,N/A,N/A
4,Ne Zha 2,2025,980477,"Animation, Fantasy, Adventure, Action",Jiao Zi,"Lu Yanting, Joseph, Han Mo, Chen Hao, Lu Qi",2025-01-29,144,80.0,2150.000000,8.068,N/A,N/A


In [22]:
fact_table.shape

(50, 13)

In [23]:
# --- Final TMDB fetch (used in final pipeline) ---
# def final_tmdb_version():


# Import necessary libraries
# - requests: For making HTTP requests to APIs
# - pandas: For handling data in DataFrames (like tables)
# - time: For adding delays to avoid overwhelming servers
import requests
import pandas as pd
import time

# Configuration settings

TMDB_API_KEY = '2762321c463faabbba42b561409706aa'
OMDB_API_KEY = '45e74674'

# Module 1: Fetch top movies from TMDB (sorted by revenue.desc)
# This function uses TMDB's discover/movie endpoint to get top-grossing films.
# It returns a DataFrame with columns: Title, Year, TMDB_ID
def fetch_top_movies_from_tmdb(num_movies=1000, min_year=2010):
    """
    Fetch top-grossing movies from TMDB using discover/movie sorted by revenue.desc.

    This function:
    1. Constructs the API URL with sort_by=revenue.desc and pagination.
    2. Optionally filters by minimum year (e.g., primary_release_year.gte=2010).
    3. Extracts title, year (from release_date), and ID for each movie.
    4. Stops once num_movies is reached.

    Args:
        num_movies (int): How many top movies to fetch (default 50).
        min_year (int): Optional minimum release year (e.g., 2010 for recent data).

    Returns:
        pd.DataFrame: A table with basic movie info (Title, Year, TMDB_ID).
    """
    movies = []
    page = 1
    while len(movies) < num_movies:
        # Construct URL with sort_by=revenue.desc
        url = f"https://api.themoviedb.org/3/discover/movie?api_key={TMDB_API_KEY}&sort_by=revenue.desc&page={page}"
        if min_year:
            url += f"&primary_release_year.gte={min_year}"  # Filter for recent years if specified
        response = requests.get(url).json()

        if 'results' not in response or not response['results']:
            break  # No more results

        for movie in response['results']:
            year = movie.get('release_date', '')[:4]  # Extract year from release_date
            if year:  # Skip if no release date
                movies.append({
                    'Title': movie['title'],
                    'Year': year,
                    'TMDB_ID': movie['id']
                })
            if len(movies) >= num_movies:
                break

        page += 1
        time.sleep(1)  # Delay to avoid rate limits

    if not movies:
        print("No movies fetched from TMDB. Check API key or parameters.")
        return pd.DataFrame()

    return pd.DataFrame(movies)

# Module 2: Fetch details from TMDB for a single movie using ID
# This function queries TMDB's movie/{id} endpoint for details.
def fetch_tmdb_details(movie_id):
    """
    Fetch movie details from TMDB API for a single movie using its ID.

    This function:
    1. Fetches detailed info including credits (cast and crew).
    2. Extracts features like genre, director, actors, release_date, runtime, budget, revenue (as Worldwide_Gross), vote_average.

    Args:
        movie_id (int): TMDB movie ID.

    Returns:
        dict: A dictionary with TMDB features (or 'N/A' if not found).
    """
    # Default dictionary with 'N/A' values
    movie_data = {
        'Genre': 'N/A',
        'Director': 'N/A',
        'Actors': 'N/A',
        'Release_Date': 'N/A',
        'Runtime': 'N/A',
        'Budget': 'N/A',
        'Worldwide_Gross': 'N/A',  # Use revenue as gross
        'Vote_Average': 'N/A'
    }

    try:
        # Construct details URL with credits appended
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&append_to_response=credits"
        details_response = requests.get(details_url).json()

        if 'status_code' in details_response and details_response['status_code'] != 200:
            raise Exception(details_response.get('status_message', 'Unknown error'))

        # Extract genres as comma-separated string
        movie_data['Genre'] = ', '.join([genre['name'] for genre in details_response.get('genres', [])])

        # Find director from crew list
        director = next((member['name'] for member in details_response['credits'].get('crew', []) if member['job'] == 'Director'), 'N/A')
        movie_data['Director'] = director

        # Get top 5 actors from cast
        actors = details_response['credits'].get('cast', [])[:5]
        movie_data['Actors'] = ', '.join([actor['name'] for actor in actors])

        # Other details
        movie_data['Release_Date'] = details_response.get('release_date', 'N/A')
        movie_data['Runtime'] = details_response.get('runtime', 'N/A')
        movie_data['Budget'] = details_response.get('budget', 'N/A')
        movie_data['Worldwide_Gross'] = details_response.get('revenue', 'N/A')  # Revenue as gross
        movie_data['Vote_Average'] = details_response.get('vote_average', 'N/A')

        time.sleep(1)  # Delay to avoid hitting API rate limits
        return movie_data
    except Exception as e:
        print(f"Error with TMDB for movie ID {movie_id}: {e}")
        return movie_data

# This function applies fetch_tmdb_details to all rows in a DataFrame using TMDB_ID
def enhance_with_tmdb(df):
    """
    Enhance the entire DataFrame with TMDB details using TMDB_ID.

    This loops through each row, fetches details by ID, and adds them to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame with 'TMDB_ID'.

    Returns:
        pd.DataFrame: DataFrame with added TMDB columns.
    """
    tmdb_data = []
    for _, row in df.iterrows():
        details = fetch_tmdb_details(row['TMDB_ID'])
        tmdb_data.append(details)

    tmdb_df = pd.DataFrame(tmdb_data)
    return pd.concat([df, tmdb_df], axis=1)

# Module 3: Fetch IMDB details using OMDb API
def fetch_imdb_details(title, year):
    """
    Fetch IMDB rating and votes using OMDb API.

    Args:
        title (str): Movie title.
        year (str): Release year.

    Returns:
        dict: Dictionary with IMDB rating and votes (or 'N/A').
    """
    try:
        # Construct OMDb URL
        url = f"http://www.omdbapi.com/?t={title}&y={year}&apikey={OMDB_API_KEY}"
        response = requests.get(url).json()

        if response['Response'] == 'True':
            return {
                'IMDB_Rating': response.get('imdbRating', 'N/A'),
                'Votes': response.get('imdbVotes', 'N/A')
            }
        else:
            print(f"OMDb error for {title}: {response.get('Error', 'Unknown')}")
            return {'IMDB_Rating': 'N/A', 'Votes': 'N/A'}

        time.sleep(1)  # Delay for rate limiting
    except Exception as e:
        print(f"Error with OMDb for {title}: {e}")
        return {'IMDB_Rating': 'N/A', 'Votes': 'N/A'}

# This function applies fetch_imdb_details to all rows
def enhance_with_imdb(df):
    """
    Enhance the DataFrame with IMDB details using OMDb API.

    Args:
        df (pd.DataFrame): Input DataFrame with 'Title' and 'Year'.

    Returns:
        pd.DataFrame: DataFrame with added IMDB columns.
    """
    imdb_data = []
    for _, row in df.iterrows():
        details = fetch_imdb_details(row['Title'], row['Year'])
        imdb_data.append(details)

    imdb_df = pd.DataFrame(imdb_data)
    return pd.concat([df, imdb_df], axis=1)

# Module 4: Clean the data
# Converts financial columns to numeric values in millions.
def clean_data(df):
    """
    Clean the DataFrame by converting financials to numbers (in millions).

    This handles:
    - Converting Worldwide_Gross, Budget, and Revenue (if present) to floats divided by 1,000,000.
    - Handles 'N/A' as NaN.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    df['Worldwide_Gross'] = pd.to_numeric(df['Worldwide_Gross'], errors='coerce') / 1000000
    df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce') / 1000000
    return df

# Module 5: Build the full fact table
# This orchestrates everything: fetch top movies, enhance with details, clean, save.
# Min Movies limited to 1000 due to oMDB having a limit of 1000 movies per API key per day
def build_fact_table(num_movies=1000, min_year=2010, use_imdb=True):
    """
    Build the complete fact table using only TMDB.

    Steps:
    1. Fetch top movies from TMDB (sorted by revenue.desc, optional min_year filter).
    2. Add TMDB details using movie IDs.
    3. Optionally add IMDB details using OMDb API.
    4. Clean the data.
    5. Save to CSV.

    Args:
        num_movies (int): Number of movies to process.
        min_year (int): Minimum release year (default 2010 for recent data; set to None for all time).
        use_imdb (bool): Whether to include IMDB data (default True).

    Returns:
        pd.DataFrame: The final fact table.
    """
    # Step 1: Get top movies from TMDB
    df = fetch_top_movies_from_tmdb(num_movies, min_year)
    if df.empty:
        return df
    # Step 2: Add TMDB details
    df = enhance_with_tmdb(df)
    # Step 3: Add IMDB if requested (using OMDb)
    if use_imdb:
        df = enhance_with_imdb(df)
    # Step 4: Clean
    df = clean_data(df)

    # Save the DataFrame to a CSV file
    df.to_csv('movie_fact_table.csv', index=False)
    print("Fact table saved to movie_fact_table.csv")
    return df

# Run the script when executed directly
# This is the entry point: calls build_fact_table and prints the first few rows
# Adjust min_year as needed (e.g., None for all time, or 2000 for broader data)
if __name__ == "__main__":
    fact_table = build_fact_table(min_year=2010)  # Example: data from 2010 onward
    print(fact_table.head())

OMDb error for Avatar: Request limit reached!
OMDb error for Avengers: Endgame: Request limit reached!
OMDb error for Avatar: The Way of Water: Request limit reached!
OMDb error for Titanic: Request limit reached!
OMDb error for Ne Zha 2: Request limit reached!
OMDb error for Star Wars: The Force Awakens: Request limit reached!
OMDb error for Avengers: Infinity War: Request limit reached!
OMDb error for Spider-Man: No Way Home: Request limit reached!
OMDb error for Inside Out 2: Request limit reached!
OMDb error for Jurassic World: Request limit reached!
OMDb error for The Lion King: Request limit reached!
OMDb error for The Avengers: Request limit reached!
OMDb error for Furious 7: Request limit reached!
OMDb error for Top Gun: Maverick: Request limit reached!
OMDb error for Frozen II: Request limit reached!
OMDb error for Barbie: Request limit reached!
OMDb error for Avengers: Age of Ultron: Request limit reached!
OMDb error for The Super Mario Bros. Movie: Request limit reached!
OMD

In [24]:
fact_table.head()


,Title,Year,TMDB_ID,Genre,Director,Actors,Release_Date,Runtime,Budget,Worldwide_Gross,Vote_Average,IMDB_Rating,Votes
0,Avatar,2009,19995,"Action, Adventure, Fantasy, Science Fiction",James Cameron,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...",2009-12-15,162,237.0,2923.706026,7.594,N/A,N/A
1,Avengers: Endgame,2019,299534,"Adventure, Science Fiction, Action",Anthony Russo,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",2019-04-24,181,356.0,2799.439100,8.237,N/A,N/A
2,Avatar: The Way of Water,2022,76600,"Action, Adventure, Science Fiction",James Cameron,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...",2022-12-14,192,350.0,2353.096253,7.609,N/A,N/A
3,Titanic,1997,597,"Drama, Romance",James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",1997-11-18,194,200.0,2264.162353,7.903,N/A,N/A
4,Ne Zha 2,2025,980477,"Animation, Fantasy, Adventure, Action",Jiao Zi,"Lu Yanting, Joseph, Han Mo, Chen Hao, Lu Qi",2025-01-29,144,80.0,2150.000000,8.068,N/A,N/A


In [25]:
fact_table.shape

(1000, 13)

In [26]:
from google.colab import files
files.download('movie_fact_table.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
import sklearn

In [29]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Part 1a: Loading and Cleaning Data
def load_and_clean_data(file_path):
    """
    Load the CSV and clean relevant columns.
    - Converts strings to numbers where possible.
    - Handles dates to extract month.
    """
    df = pd.read_csv(file_path)
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert to number, ignore errors
    df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')
    df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce')
    df['Worldwide_Gross'] = pd.to_numeric(df['Worldwide_Gross'], errors='coerce')
    df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors='coerce')  # Convert to date
    df['Release_Month'] = df['Release_Date'].dt.month.fillna(0)  # Get month, fill missing with 0
    return df

# Part 1b: Engineering Features
def engineer_features(df):
    """
    Engineer features like genre encoding, director encoding, and num_actors.
    - Splits genres into lists and encodes them.
    - Encodes top directors.
    - Counts number of actors.
    Returns features, target, mlb (genre encoder), and top_directors.
    """
    # Handling genres: Split comma-separated into lists
    df['Genre_List'] = df['Genre'].apply(lambda x: [g.strip() for g in str(x).split(',')] if pd.notnull(x) else [])

    # Multi-label encode genres (turns genres into binary columns, e.g., Action: 1 or 0)
    mlb = MultiLabelBinarizer()
    genre_encoded = pd.DataFrame(mlb.fit_transform(df['Genre_List']), columns=mlb.classes_, index=df.index)

    # Director: One-hot encode top 10, group others (turns directors into binary columns)
    top_directors = df['Director'].value_counts().nlargest(10).index
    df['Director'] = df['Director'].apply(lambda x: x if x in top_directors else 'Other')
    director_encoded = pd.get_dummies(df['Director'], prefix='Director')

    # Num_Actors: Count from Actors column
    df['Num_Actors'] = df['Actors'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)

    # Combine all features into one DataFrame
    features = pd.concat([df[['Year', 'Runtime', 'Budget', 'Release_Month', 'Num_Actors']], genre_encoded, director_encoded], axis=1)
    target = df['Worldwide_Gross']

    # Drop rows with missing target or budget (key feature)
    valid_idx = target.notnull() & features['Budget'].notnull()
    features = features[valid_idx].fillna(features.median(numeric_only=True))  # Fill missing numbers with medians
    target = target[valid_idx]

    return features, target, mlb, top_directors

In [46]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score

# Part 2a: Training Model
def train_model(X_train, y_train):
    """
    Train the Random Forest model.
    - n_estimators=100: Builds 100 trees.
    - random_state=42: For reproducible results.
    """
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)  # Fit the model to training data
    return model

# Part 2b: Evaluating the Model
def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model on test data.
    - Predicts on test set.
    - Computes R² score (how well the model explains variance; 1 is perfect).
    """
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print("Test R²:", r2)
    return r2

# Part 2c: Evaluate with K-Fold Cross-Validation
def evaluate_model_with_kfold(X, y, n_estimators=100, k=5):
    """
    Evaluate the model using K-Fold Cross-Validation.
    - Uses KFold to split data into k folds.
    - Trains and evaluates the Random Forest model k times.
    - Computes average R² across folds.
    """
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')
    print("R² scores for each fold:", scores)
    print("Average R²:", scores.mean())
    print("Standard Deviation of R²:", scores.std())
    return scores.mean()


In [47]:
# Part 3: Preparing New Movie Input
def prepare_new_movie_input(movie_details, mlb, top_directors, training_columns):
    """
    Prepare features for a new movie.
    - Uses the same encoding as training.

    Args:
        movie_details (dict): Dictionary with movie data.
        mlb: MultiLabelBinarizer from training.
        top_directors: List of top directors from training.
        training_columns (list): List of feature columns from training.

    Returns:
        pd.DataFrame: Prepared features.
    """
    input_df = pd.DataFrame([movie_details])

    # Genre encoding (using mlb from training)
    genre_encoded = pd.DataFrame(mlb.transform([movie_details['Genre_List']]), columns=mlb.classes_)

    # Director encoding
    input_df['Director'] = input_df['Director'].apply(lambda x: x if x in top_directors else 'Other')
    director_encoded = pd.get_dummies(input_df['Director'], prefix='Director')

    # Combine features
    all_features = pd.concat([input_df[['Year', 'Runtime', 'Budget', 'Release_Month', 'Num_Actors']], genre_encoded, director_encoded], axis=1)

    # Reindex to match training columns, fill missing with 0 (e.g., unseen genres/directors get 0)
    all_features = all_features.reindex(columns=training_columns, fill_value=0)

    return all_features

In [54]:
# Part 4: Predicting Revenue
def predict_revenue(model, input_features):
    """
    Predict revenue using the trained model.
    """
    prediction = model.predict(input_features)[0]  # Get the first (only) prediction
    return prediction

In [55]:
# Run Everything Sequentially
# Load and prepare data
df = load_and_clean_data('movie_fact_table.csv')
features, target, mlb, top_directors = engineer_features(df)

# Get training columns for later use
training_columns = features.columns.tolist()

# Split data for training
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

# Train and evaluate
model = train_model(X_train, y_train)
evaluate_model(model, X_test, y_test)

# Movie details for Avatar: Fire and Ash
movie_details = {
    'Year': 2025,
    'Runtime': 195,
    'Budget': 400,
    'Release_Month': 12,
    'Num_Actors': 5,
    'Genre_List': ['Action', 'Adventure', 'Fantasy', 'Science Fiction'],
    'Director': 'James Cameron'
}

# Prepare input
all_features = prepare_new_movie_input(movie_details, mlb, top_directors, training_columns)

# Predict
prediction = predict_revenue(model, all_features)
print("Predicted Worldwide Gross: ${:.3f} billion".format(prediction / 1000))  # Format in billions

Test R²: 0.37052691736567367
Predicted Worldwide Gross: $1.930 billion
